<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# FlatLay Generator V2 - Full Auto Version

# --- Bước 0: Cài đặt thư viện cần thiết ---
!pip install diffusers transformers accelerate controlnet_aux --upgrade

# --- Bước 1: Mount Google Drive và tạo Folder Tự Động ---
from google.colab import drive
import os

drive.mount('/content/drive')

input_folder = "/content/drive/MyDrive/FlatLay_Input"
output_folder = "/content/drive/MyDrive/FlatLay_Generated"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# --- Bước 2: Cho upload nếu chưa có file ---
from google.colab import files

if len(os.listdir(input_folder)) == 0:
    print("\u2728 Chưa có ảnh input. Chị vui lòng upload file (.jpg/.png/.jpeg/.webp) bên dưới nhé!")
    uploaded = files.upload()
    for filename in uploaded.keys():
        with open(os.path.join(input_folder, filename), 'wb') as f:
            f.write(uploaded[filename])
    print(f"\u2728 Đã upload {len(uploaded)} file vào {input_folder}")

# --- Bước 3: Import thư viện xử lý ---
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# --- Bước 4: Load model ---
os.environ["DIFFUSERS_FLAX_AVAILABLE"] = "0"
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

canny = CannyDetector()

# --- Bước 5: Hàm process FlatLay ---
def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path).convert("RGB")
    image = image.resize((512, 512))

    canny_image = canny(image)

    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    generated_image.save(output_path)

# --- Bước 6: Gen Prompt FlatLay Chuẩn Pinterest ---
def generate_prompt(image_name):
    base_prompt = "flat lay of a full casual outfit on white background, minimalist aesthetic, pinterest style, natural lighting, no shadows, realistic photo look, beige and neutral tones, including: "

    # Dựa vào tên file, đoán định category
    name = image_name.lower()
    if "jeans" in name:
        base_prompt += "blue jeans, white t-shirt, sneakers, accessories like tote bag, sunglasses, simple jewelry"
    elif "beach" in name or "bikini" in name:
        base_prompt += "beachwear set, bikini, flip flops, beach tote, sunglasses, summer bracelet"
    elif "skirt" in name:
        base_prompt += "denim skirt, crop top, sneakers, sunglasses, casual accessories"
    else:
        base_prompt += "casual clothing set, jacket, t-shirt, jeans, sneakers, accessories"
    return base_prompt

# --- Bước 7: Quét toàn bộ folder và xử lý ---
has_input = False
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.webp')):
        has_input = True
        input_image_path = os.path.join(input_folder, filename)

        output_image_name = filename.split('.')[0] + "_flatlay.png"
        output_image_path = os.path.join(output_folder, output_image_name)

        prompt = generate_prompt(filename)

        process_flatlay(input_image_path, prompt, output_image_path)
        print(f"\u2705 Đã gen: {output_image_name}")

if not has_input:
    print(f"\u26a1\ufe0f Chưa có ảnh input. Upload ảnh và chạy lại.")
else:
    print("\ud83c\udf3f\u2728 Flat Lay Generation Complete!")
